

> **Conversation Management & Classification using Groq API**

>




Implementing two core tasks using Groq APIs -

*Task 1:* Managing Conversation History with Summarization

*Task 2:* JSON Schema Classification & Information Extraction

In [1]:
# Step 1: Install the required library
!pip install groq -q

# Step 2: Import necessary libraries
import os
import json
from groq import Groq
from google.colab import userdata

print("Libraries installed and imported successfully.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.9 MB/s eta 0:00:00
Libraries installed and imported successfully.


In [2]:
# Configure the Groq API Client
try:
    # API key from Colab secrets
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("GROQ_API_KEY has been set successfully.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'GROQ_API_KEY' not found.")
    print("Please go to the 'Secrets' tab (key icon on the left) and add your Groq API key.")
except Exception as e:
    print(f"An error occurred: {e}")

# Initialize Groq client
try:
    client = Groq()
    print("Groq client initialized.")
except Exception as e:
    print(f"Failed to initialize Groq client: {e}")


GROQ_API_KEY has been set successfully.
Groq client initialized.


In [3]:
# ========================
# Model Configuration
# ========================
DEFAULT_CHAT_MODEL = "llama-3.1-8b-instant"
DEFAULT_FUNC_MODEL = "llama-3.3-70b-versatile"


In [4]:
# Task 1: Managing Conversation History with Summarization
class ConversationManager:
    """
    Manages conversation history with features for periodic summarization and truncation.
    """
    def __init__(self, model=DEFAULT_CHAT_MODEL, summarize_every_k_runs=5):
        self.model = model
        self.conversation_history = []
        self.run_count = 0
        self.summarize_every_k = summarize_every_k_runs

    def add_message(self, role, content):
        """Adds a new message to the history and handles periodic summarization."""
        self.conversation_history.append({"role": role, "content": content})

        if role == 'assistant':
            self.run_count += 1
            if self.run_count % self.summarize_every_k == 0 and len(self.conversation_history) > 1:
                print(f"\n--- Triggering summarization after {self.run_count} runs. ---")
                self._summarize()

    def _summarize(self):
        """Internal method to summarize the current history and replace it."""
        history_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.conversation_history])

        try:
            summary_response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a conversation summarizer. Summarize the following dialogue concisely, capturing the key points and context for a future AI interaction."},
                    {"role": "user", "content": history_str}
                ],
                temperature=0.2,
            )
            summary_text = summary_response.choices[0].message.content
            self.conversation_history = [{"role": "system", "content": f"Summary of previous conversation: {summary_text}"}]
            print("--- Conversation summarized and history replaced. ---")
        except Exception as e:
            print(f"Could not summarize the conversation. Error: {e}")

    def get_history(self):
        return self.conversation_history

    def get_truncated_history_by_turns(self, n_turns):
        return self.conversation_history[-(n_turns * 2):]

    def get_truncated_history_by_length(self, max_chars):
        truncated_history = []
        current_length = 0
        for message in reversed(self.conversation_history):
            msg_len = len(message['content'])
            if current_length + msg_len > max_chars:
                break
            truncated_history.insert(0, message)
            current_length += msg_len
        return truncated_history


In [12]:
print("\n" + "="*80)
print("DEMONSTRATION OF TASK 1: CONVERSATION HISTORY MANAGEMENT & SUMMARIZATION")
print("="*80 + "\n")

manager = ConversationManager(summarize_every_k_runs=5)

conversation_flow = [
    ("user", "Hi, I’ve been hearing a lot about mental health struggles after COVID. Is it really such a big issue?"),
    ("assistant", "Yes, it is. WHO reported a 25% global increase in anxiety and depression cases during the pandemic."),
    ("user", "That’s shocking. Which groups were most affected?"),
    ("assistant", "Young adults, healthcare workers, and people who lost jobs were among the hardest hit."),
    ("user", "So how does AI come into this picture?"),
    ("assistant", "AI tools are now used to screen symptoms, run mental health chatbots, and even analyze speech for early signs of depression."),
    ("user", "Do these chatbots really help?"),
    ("assistant", "Yes, apps like Woebot and Wysa have shown reduced anxiety symptoms in just two weeks of use."),
    ("user", "But can AI really replace a therapist?"),
    ("assistant", "Not replace, but support. They provide 24/7 access while human therapists handle deeper treatment."),
    ("user", "How accurate are these AI tools in detecting depression?"),
    ("assistant", "Some voice-analysis models have achieved 80–85% accuracy in detecting depression from tone and word choice."),
    ("user", "That sounds promising, but what about privacy?"),
    ("assistant", "Privacy is a major concern. Data must comply with HIPAA and GDPR, and explainable AI is encouraged."),
    ("user", "Explainable AI? What does that mean?"),
    ("assistant", "It means the AI should show why it made a prediction, for example pointing to slower speech or negative phrasing as signals."),
    ("user", "Are hospitals using this technology too?"),
    ("assistant", "Yes, some hospitals use predictive AI to flag patients at risk of suicide, with sensitivity around 78%."),
    ("user", "That’s powerful, but what if it makes mistakes?"),
    ("assistant", "Exactly why doctors still make the final call. AI should be an assistant, not the sole decision maker."),
    ("user", "What about bias in AI models?"),
    ("assistant", "Good question. If training data underrepresents certain communities, predictions may be less accurate for them."),
    ("user", "So what are researchers doing about that?"),
    ("assistant", "They are diversifying datasets and building fairness checks into the AI systems."),
    ("user", "How has telemedicine grown since COVID?"),
    ("assistant", "Massively. In the U.S., telehealth visits increased by over 1500% in 2020, and AI tools are now embedded in those platforms."),
    ("user", "Does AI reduce costs compared to traditional therapy?"),
    ("assistant", "Yes, studies show AI-supported therapy can cut costs by 30–40% while expanding access."),
    ("user", "Are there any real-world success stories?"),
    ("assistant", "Yes, in the UK, an AI triage system helped reduce waiting times for mental health consultations by 25%."),
    ("user", "What’s the biggest ethical worry with AI in mental health?"),
    ("assistant", "The main concern is misuse of sensitive data and ensuring patients don’t rely solely on AI without human care."),
    ("user", "So what’s the future of AI in mental health care?"),
    ("assistant", "By 2030, AI-driven support is expected to be integrated into global primary care, helping close the treatment gap affecting 70% of patients today."),
    ("user", "Thanks for explaining all this. It’s fascinating and a bit scary at the same time."),
    ("assistant", "I understand. AI in healthcare is powerful but needs careful, ethical use to truly help people.")
]

for i, (role, content) in enumerate(conversation_flow):
    print(f"Adding message {i+1}: Role='{role}', Content='{content[:40]}...'")
    manager.add_message(role, content)
    if role == 'assistant':
        print(f"\n>>> Current History (Run {manager.run_count}):")
        print(json.dumps(manager.get_history(), indent=2))
        print("-" * 20)
print("\n" + "="*50)
print("Demonstrating Truncation Options on Final History")
print("="*50 + "\n")
truncated_by_turns = manager.get_truncated_history_by_turns(n_turns=2)
print("\n### a. Truncated by last 2 turns ###")
print(json.dumps(truncated_by_turns, indent=2))

truncated_by_length = manager.get_truncated_history_by_length(max_chars=150)
print("\n### b. Truncated by last 150 characters ###")
print(json.dumps(truncated_by_length, indent=2))



DEMONSTRATION OF TASK 1: CONVERSATION HISTORY MANAGEMENT & SUMMARIZATION

Adding message 1: Role='user', Content='Hi, I’ve been hearing a lot about mental...'
Adding message 2: Role='assistant', Content='Yes, it is. WHO reported a 25% global in...'

>>> Current History (Run 1):
[
  {
    "role": "user",
    "content": "Hi, I\u2019ve been hearing a lot about mental health struggles after COVID. Is it really such a big issue?"
  },
  {
    "role": "assistant",
    "content": "Yes, it is. WHO reported a 25% global increase in anxiety and depression cases during the pandemic."
  }
]
--------------------
Adding message 3: Role='user', Content='That’s shocking. Which groups were most ...'
Adding message 4: Role='assistant', Content='Young adults, healthcare workers, and pe...'

>>> Current History (Run 2):
[
  {
    "role": "user",
    "content": "Hi, I\u2019ve been hearing a lot about mental health struggles after COVID. Is it really such a big issue?"
  },
  {
    "role": "assistant",
   

In [6]:
# JSON schema for information extraction
user_details_schema = {
    "type": "object",
    "properties": {
        "name": {"type": ["string", "null"], "description": "The full name of the user."},
        "email": {"type": ["string", "null"], "description": "The user's email address."},
        "phone": {"type": ["string", "null"], "description": "The user's phone number, including country code if available."},
        "location": {"type": ["string", "null"], "description": "The user's city and country, e.g., 'San Francisco, USA'."},
        "age": {"type": ["integer", "null"], "description": "The user's age as an integer."}
    },
    "required": ["name", "email", "phone", "location", "age"],
    "additionalProperties": False
}

# Tool for Groq API
extraction_tool = {
    "type": "function",
    "function": {
        "name": "extract_user_details",
        "description": "Extracts user information from the chat text and returns it in a structured JSON format. If a piece of information is not found, it should be set to null.",
        "parameters": user_details_schema
    },
}


In [7]:
def extract_info_from_chat(user_input):
    """
    Uses Groq's tool-calling feature to parse user input against a schema.
    """
    print(f"Parsing chat: '{user_input}'")
    try:
        response = client.chat.completions.create(
            model=DEFAULT_FUNC_MODEL,
            messages=[
                {"role": "system", "content": "You are an information extraction assistant. Your task is to extract user details from the provided text and format it using the 'extract_user_details' tool. Only use the tool provided."},
                {"role": "user", "content": user_input}
            ],
            tools=[extraction_tool],
            tool_choice={"type": "function", "function": {"name": "extract_user_details"}},
            temperature=0.0
        )

        response_message = response.choices[0].message
        if response_message.tool_calls:
            tool_call = response_message.tool_calls[0]
            if tool_call.function.name == "extract_user_details":
                extracted_data = json.loads(tool_call.function.arguments)
                return extracted_data
        return {"error": "Tool not called by the model."}

    except Exception as e:
        return {"error": f"An API call error occurred: {e}"}


In [9]:
print("\n" + "="*80)
print("DEMONSTRATION OF TASK 2: JSON SCHEMA CLASSIFICATION & INFORMATION EXTRACTION")
print("="*80 + "\n")

sample_chats = [
    "Hello, my name is Sarah Thompson and I'm 32. I live in Boston, USA. I recently tried an AI-based mental health app to manage my anxiety post-COVID. My email is sarah.t@example.com and my phone is +1-617-555-1010.",
    "Hi, I'm Rajesh Kumar, 28 years old, based in Mumbai, India. After recovering from COVID, I participated in a hospital's AI-driven monitoring program for long-COVID symptoms. My email is rajesh.kumar@email.com.",
    "Greetings, my name is Emily Wong from Toronto, Canada. I’ve been using an AI-assisted teletherapy platform to cope with depression caused by post-COVID isolation. My phone number is +1-416-555-2323.",
    "Hey, I'm 38, living in Los Angeles, USA. I enrolled in an AI mental health assessment study after experiencing sleep disturbances post-COVID. Reach me at m.lopez@email.com or call +1-310-555-7788.",
    "I'm exploring AI-driven chatbots and apps designed for post-COVID stress and anxiety management, but I don't want to share my personal details at the moment."
]

for i, chat in enumerate(sample_chats):
    print(f"\n--- Parsing Chat {i+1} ---")
    extracted_info = extract_info_from_chat(chat)

    print("\n>>> Extracted Information:")
    if "error" in extracted_info:
        print(f"   Error: {extracted_info['error']}")
    else:
        print(json.dumps(extracted_info, indent=4))
        if all(key in extracted_info for key in user_details_schema["properties"]):
            print("\n   Validation: SUCCESS - Output matches the required schema keys.")
        else:
            print("\n   Validation: FAILED - Output is missing schema keys.")



DEMONSTRATION OF TASK 2: JSON SCHEMA CLASSIFICATION & INFORMATION EXTRACTION


--- Parsing Chat 1 ---
Parsing chat: 'Hello, my name is Sarah Thompson and I'm 32. I live in Boston, USA. I recently tried an AI-based mental health app to manage my anxiety post-COVID. My email is sarah.t@example.com and my phone is +1-617-555-1010.'

>>> Extracted Information:
{
    "age": 32,
    "email": "sarah.t@example.com",
    "location": "Boston, USA",
    "name": "Sarah Thompson",
    "phone": "+1-617-555-1010"
}

   Validation: SUCCESS - Output matches the required schema keys.

--- Parsing Chat 2 ---
Parsing chat: 'Hi, I'm Rajesh Kumar, 28 years old, based in Mumbai, India. After recovering from COVID, I participated in a hospital's AI-driven monitoring program for long-COVID symptoms. My email is rajesh.kumar@email.com.'

>>> Extracted Information:
{
    "age": 28,
    "email": "rajesh.kumar@email.com",
    "location": "Mumbai, India",
    "name": "Rajesh Kumar",
    "phone": null
}

   Validat

In [31]:
!git config --global user.name "Divya Marangappanavar"
!git config --global user.email "divyamarangappanavar@gmail.com"

In [43]:
!rm -rf .git

In [44]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [45]:
!git add "Conversation Management & Classification using Groq API.ipynb"

fatal: pathspec 'Conversation Management & Classification using Groq API.ipynb' did not match any files
